In [ ]:
!pip install sentence_transformers

In [ ]:
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
import pandas as pd
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
embedding_model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [ ]:
def generate_embedding(sentence):
    tokens = tokenizer.tokenize(sentence)
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)])

    with torch.no_grad():
        outputs = embedding_model(input_ids)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

    return embedding.tolist()

In [ ]:
def preprocess_csv_dataset(csv_path):
    preprocessed_rows = []

    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            row_embeddings = [generate_embedding(sentence) for sentence in row]
            preprocessed_rows.append(row_embeddings)

    return preprocessed_rows

In [ ]:
def find_most_similar_row(input_sentence, preprocessed_csv_data):
    input_embedding = generate_embedding(input_sentence)

    max_similarity = float('-inf')
    best_row_idx = -1

    for idx, row in enumerate(preprocessed_csv_data):
        for sent_embedding in row:
          similarity = cosine_similarity([input_embedding], [sent_embedding])[0][0]
          if similarity > max_similarity:
              max_similarity = similarity
              best_row_idx = idx

    return best_row_idx

In [ ]:
def get_first_sentence_of_row(csv_path, row_idx):
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        for idx, row in enumerate(reader):
            if idx == row_idx:
                return row[0]

In [ ]:
def get_answer(csv_path, row_idx):
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        for idx, row in enumerate(reader):
            if idx == row_idx:
                return row[1]

In [ ]:
def print_answer(input_sentence):
    row_index = find_most_similar_row(input_sentence, preprocessed_data)
    row_sentence = get_first_sentence_of_row(csv_path, row_index)
    answer = get_answer(answer_path, row_index)
    print(f'row: {row_index} \t question: {row_sentence} \n answer: {answer} \n')

In [ ]:
csv_path = "./questions_dataset_final_persian.csv"
answer_path = "./SATCorpus_persian.csv"
preprocessed_data = preprocess_csv_dataset(csv_path)

In [ ]:
input_sentences = [
    'تو کیستی؟',
    'تمرین سوم',
    'تمرین هفت چیست؟',
    'تکلیف درس چهارم را بگو',
    'راجب درس ۵ توضیح بده',
    'چگونه با کودک می‌توانم ارتباط بگیرم؟',
    'سلام',
    'چرا باید با صدای بلند با کودک صبحت کنم؟',
    'چرا باید به طبیعت دلبست؟'
]
for tmp_sentence in input_sentences:
    print_answer(tmp_sentence)

row: 147 	 question: تو کی هستی؟ 
 answer: من یک معلم مجازی بر اساس صدا هستم که برای آموزش تکنیک پیوستن به خود (SAT) طراحی شده‌ام. شما می‌توانید سوالاتی را از طریق SAT از من بپرسید یا دوره آموزشی SAT را شروع کنید. 

row: 69 	 question: تمرین ۳ 
 answer: تمرین ۳: خواندن یک ترانه از عشق و احترام. نسخه‌هایی از عکس شاد خود را چاپ کنید تا در خانه، محل کار یا کیف پولتان قرار دهید. حتی می‌توانید آن را به عنوان پس زمینه گوشی یا لپ‌تاپ‌تان تنظیم کنید. سپس، یک ترانه شاد، مطربانه و محبوب که احساسات دلچسب و احترام را در شما برانگیخته است انتخاب کنید. این ترانه را به حفظ بسپارید و همچنان که می‌توانید در روزانه خود اجرا کنید. هنگامی که به عکس شاد نگاه می‌کنید، با خواندن این ترانه ارتباط عاطفی عمیقی با کودک ذهنتان برقرار کنید. به‌تدریج شروع به آرام کردن صدای خود کنید و با گذشت زمان، اجازه دهید صدایتان بلندتر شود و بیشتر از بدن‌تان استفاده کنید (مثلاً با تکان دادن شانه‌ها، دست‌ها و جابه‌جا کردن ابروهایتان). در همین حال، تصور کنید که به عنوان یک والد، در گفت‌وگوی محبت‌آمیز و پرشور با کودک رقص می‌کنید و

In [ ]:
extended_synonyms_dict = {
    "توضیح دهید": ["شرح دهید", "تفصیل بیاورید", "روشن‌سازی کنید", "راهنمایی کنید"],
    "آیا می‌توانید": ["آیا ممکن است که", "آیا می‌توانیم از شما بخواهیم که", "آیا قابل استفاده که", "آیا شما قادرید که"],
    "چگونه": ["به چه شکلی", "به چه ترتیبی", "به چه روشی", "به چه طریقه‌ای"],
    "نظریه": ["پارادایم", "فرضیه", "رویکرد", "مدل"],
    "دلبستگی": ["محبت", "علاقه", "رابطه", "پیوند"],
    "رفتارها": ["کنش‌ها", "عمل‌ها", "فعالیت‌ها", "واکنش‌ها"],
    "حس": ["احساس", "انگیزه", "تجربه", "ادراک"],
    "امنیت": ["ایمنی", "پایداری", "ثبات", "محافظت"],
    "تغییر": ["تحول", "تبدیل", "تجدید", "دگرگونی"],
    "انواع": ["گونه‌ها", "نوعیت‌ها", "دسته‌ها", "شاخصه‌ها"],
    "بزرگسالان": ["بالغین", "افراد بالغ", "نسل بزرگتر", "جوانان"],
    "کودکی": ["طفولیت", "دوران کودکی", "سال‌های اولیه", "دوران کوچکی"],
    "نوجوانی": ["دوران نوجوانی", "سن نوجوانی", "دوره نوجوانی", "سال‌های نوجوانی"],
    "۱": ["یک", "۱", "اول"],
    "۲": ["دو", "۲", "دوم"],
    "۳": ["سه", "۳", "سوم"],
    "۴": ["چهار", "۴", "چهارم"],
    "۵": ["پنج", "۵", "پنجم"],
    "۶": ["شش", "۶", "ششم"],
    "۷": ["هفت", "۷", "هفتم"],
    "۸": ["هشت", "۸", "هشتم"],
    "۹": ["نه", "۹", "نهم"],
    "۱۰": ["ده", "۱۰", "دهم"],
    "۱۱": ["یازده", "۱۱", "یازدهم"],
    "۱۲": ["دوازده", "۱۲", "دوازدهم"],
    "۱۳": ["سیزده", "۱۳", "سیزدهم"],
    "۱۴": ["چهارده", "۱۴", "چهاردهم"],
    "۱۵": ["پانزده", "۱۵", "پانزدهم"],
    "۱۶": ["شانزده", "۱۶", "شانزدهم"],
    "۱۷": ["هفده", "۱۷", "هفدهم"],
    "۱۸": ["هجده", "۱۸", "هجدهم"],
    "۱۹": ["نوزده", "۱۹", "نوزدهم"],
    "۲۰": ["بیست", "۲۰", "بیستم"],
    "۲۱": ["بیست و یک", "۲۱", "بیست و یکم"],
    "۲۲": ["بیست و دو", "۲۲", "بیست و دوم"],
    "۲۳": ["بیست و سه", "۲۳", "بیست و سوم"],
    "۲۴": ["بیست و جهار", "۲۴", "بیست و چهارم"],
    "۲۵": ["بیست و پنج", "۲۵", "بیست و پنجم"],
    "۲۶": ["بیست و شش", "۲۶", "بیست و ششم"]
}

extended_starters = ["آیا می‌توانید بیشتر توضیح دهید که ", "چگونه ", "آیا می‌توانید بگویید ", "می‌توانید توضیح دهید که ", "آیا قابل استفاده که ", "به چه شکلی ", "آیا ممکن است که ", "می‌توانید راهنمایی کنید که "]
extended_enders = [" چیست؟", " به چه معناست؟", " چگونه است؟", " چه اهمیتی دارد؟", " چرا مهم است؟", " چه تاثیری دارد؟", " چه کاربردی دارد؟", " چه مزایایی دارد؟"]

In [ ]:
def generate_extended_similar_questions(question, num_similar=4):
    similar_questions = []
    for _ in range(num_similar):
        starter = random.choice(extended_starters)
        ender = random.choice(extended_enders)

        modified_question = question
        num_words = len(modified_question.split())
        for word, synonyms in extended_synonyms_dict.items():
            if word in modified_question:
                word_pattern = r'\b' + re.escape(word) + r'\b'
                replacement = random.choice(synonyms)
                modified_question = re.sub(word_pattern, replacement, modified_question)
        if (num_words > 3):
            similar_questions.append(starter + modified_question + ender)
        else:
            similar_questions.append(modified_question)

    return similar_questions

In [ ]:
questions_df = pd.read_csv(csv_path)
similar_questions = []
predicted_questions = []
main_questions = []

for idx, row in questions_df.iterrows():
    generated_questions = generate_extended_similar_questions(row[0], num_similar=4)
    main_question = row[0]
    for q in generated_questions:
        row_index = find_most_similar_row(q, preprocessed_data)
        predicted_questions.append(get_first_sentence_of_row(csv_path, row_index))
        main_questions.append(main_question)
        similar_questions.append(q)
        print(main_question)
        print(q)
        print(get_first_sentence_of_row(csv_path, row_index))
        print("####################")

نظریه دلبستگی چیست؟
مدل محبت چیست؟
دلبستگی چیست؟
####################
نظریه دلبستگی چیست؟
مدل علاقه چیست؟
نمونه‌های نقش مدل‌ها و تأییدیه‌های آن‌ها
####################
نظریه دلبستگی چیست؟
مدل پیوند چیست؟
نمونه‌های نقش مدل‌ها و تأییدیه‌های آن‌ها
####################
نظریه دلبستگی چیست؟
فرضیه محبت چیست؟
نظریه دلبستگی چیست؟
####################
دلبستگی چیست؟
پیوند چیست؟
دلبستگی چیست؟
####################
دلبستگی چیست؟
پیوند چیست؟
دلبستگی چیست؟
####################
دلبستگی چیست؟
محبت چیست؟
بزرگسال چیست؟
####################
دلبستگی چیست؟
علاقه چیست؟
دلبستگی چیست؟
####################
آیا رفتارهای دلبستگی نشان‌دهنده حس امنیت هستند؟
می‌توانید توضیح دهید که آیا رفتارهای رابطه نشان‌دهنده تجربه پایداری هستند؟ چه کاربردی دارد؟
رفتار دلبستگی چیست؟
####################
آیا رفتارهای دلبستگی نشان‌دهنده حس امنیت هستند؟
آیا می‌توانید بگویید آیا رفتارهای پیوند نشان‌دهنده تجربه ثبات هستند؟ چگونه است؟
آیا رفتارهای دلبستگی نشان‌دهنده حس امنیت هستند؟
####################
آیا رفتارهای دلبستگی نشان‌دهنده حس 

In [ ]:
filename = "results.csv"
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Main Questions", "Similar Questions", "Predicted Questions"])
    for main, similar, predicted in zip(main_questions, similar_questions, predicted_questions):
        writer.writerow([main, similar, predicted])
count_equal = sum(1 for main, predicted in zip(main_questions, predicted_questions) if main == predicted)
percentage_equal = (count_equal / len(main_questions)) * 100

print(f"{percentage_equal}% of predicted questions were equal to the main questions.")

80.95238095238095% of predicted questions were equal to the main questions.


In [ ]:
import joblib
joblib.dump(preprocessed_data, "./embedded_questions.joblib")

['./embedded_questions.joblib']